In [16]:
import pandas as pd
import matplotlib as mp

%matplotlib inline

In [17]:
auctions = pd.read_csv('../../data/auctions.csv.gzip',compression='gzip',\
                        usecols=['date','device_id','platform','ref_type_id','source_id'],\
                        dtype={'device_id':'int64','platform':'category','ref_type_id':'category','source_id':'category'},\
                        parse_dates=['date'])

In [18]:
auctions['date'].min()

Timestamp('2019-03-05 00:52:33.352526')

In [19]:
auctions['date'].max()

Timestamp('2019-03-13 23:59:59.997032')

## Armado de tiempo entre arribos

Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.

In [20]:
grp = ['device_id']
#Ordeno por grp y fecha
auctions.sort_values(by=['device_id', 'date'], inplace=True)

In [21]:
min_value = 2
max_value = 150
auctions = auctions.groupby(grp, sort=False).filter(lambda data: (min_value < len(data) < max_value))

In [22]:
auctions['next_date'] = auctions.groupby(grp, as_index = False, sort=False)['date']\
                                              .transform(lambda x: x.shift(-1))
auctions = auctions.loc[(~auctions['next_date'].isnull())]
auctions['secs_to_next'] = (auctions['next_date'] - auctions['date'])\
                                        .transform(lambda x: round(x.total_seconds()))

In [23]:
filtered = auctions.loc[auctions['secs_to_next'] < 120]

In [24]:
devices = filtered.groupby(grp, as_index='False')['secs_to_next'].mean().to_frame()

In [25]:
devices.head()

,secs_to_next
device_id,
113858820194433,2.285714
148049712234927,6.000000
163367509015039,39.440000
250378692954397,8.200000
356084629798952,20.000000


## Tiempo desde ultima aparicion

In [26]:
max_date = auctions['date'].max()

In [27]:
time = auctions.groupby(grp).apply(lambda x: round((max_date - x['date'].max()).total_seconds())).to_frame()
time.columns = ['time_from_last_arrival']

In [28]:
devices = devices.merge(time, how='outer', left_index=True, right_index=True)

In [29]:
devices.head()

,secs_to_next,time_from_last_arrival
device_id,,
113858820194433,2.285714,32593
148049712234927,6.000000,760317
163367509015039,39.440000,286790
250378692954397,8.200000,139412
356084629798952,20.000000,436175


## Cantidad de apariciones en encuestas 

## Secuencia de ultimos 5 eventos del dispositivo